In [3]:
# !pip install dash
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Load the dataset
df = pd.read_csv('./dataset/Taylor_Swift_albumsales.csv')

# Create a Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Graph(id='world-heatmap', style={'height': '80vh'}),  # Set the height to 80% of the viewport height
    html.Div([
        html.Label('Select Year:'),
        dcc.Slider(
            id='year-slider',
            min=df['Year'].min(),
            max=df['Year'].max(),
            value=df['Year'].min(),
            marks={str(year): str(year) for year in df['Year'].unique()},
            step=None
        )
    ])
])

# Define callback to update the heatmap based on the selected year
@app.callback(
    Output('world-heatmap', 'figure'),
    [Input('year-slider', 'value')]
)
def update_heatmap(year):
    filtered_df = df[(df['Year'] == year) & (df['Sales'] > 0)]  # Filter out negative sales values
    fig = px.choropleth(
        filtered_df,
        locations='Country',
        locationmode='country names',
        color='Sales',
        hover_name='Country',
        color_continuous_scale='Viridis',
        color_continuous_midpoint=filtered_df['Sales'].mean(),  # Set midpoint to the mean of positive sales values
        title=f'Sales of the albums Worldwide in Year {year}'
    )
    fig.update_geos(projection_type="equirectangular")
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
